# Decision Tree Regression

## Importing the libraries

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import redshift_connector
import keyring

## Importing the dataset

In [20]:
### Importing via Redshift
pwd = keyring.get_password("redshift-production.db.customink.com", "dan.caley")

# Connecting to redshift
# https://docs.aws.amazon.com/redshift/latest/mgmt/python-connect-examples.html#python-connect-query
conn = redshift_connector.connect(
    host='redshift-production.db.customink.com',
    database='cink',
    user='dan.caley',
    password= pwd
)

# Reading SQL File
open_file = open('sql_code.sql','r')
sql_file = open_file.read()
open_file.close()

# Running Query from sql file
dataset = pd.read_sql_query(sql_file, conn)

# Removing Binary headers
remove_binary = dataset.columns.astype(str).str.replace("'b",'')
dataset.columns = remove_binary



DatabaseError: Execution failed on sql 'with tmp_errors as (
    select distinct order_id
    from factdata.errors
    where external_flag = TRUE
)


select customer_orders.customer_order_id,
       customer_orders.date_placed,
       datediff('d', customer_orders.date_placed, customer_orders.date_delivered) days_deliverd,
       customer_orders.net_price,
       customer_orders.total_units,
       count(distinct designs.design_id)                                          designs_prior_30,
       nvl2(tmp_errors.order_id, 1, 0)                                            errors,
       round(median_household_income::numeric(20) * population::numeric(20))      zip_wealth,
       segment_name_uber,
       case
           when segment_name_uber in ('Friends & Family', 'All Athletics', 'Organizations')
               then 'Family, Org, & Athletics'
           else segment_name_uber end   as                                        segment_name_ultra,
       case when segment_name_ultra = 'Fam

In [ ]:
dataset.describe()

In [ ]:
# Check NA's
dataset.isna().sum()

In [ ]:
#dataset = pd.read_csv('Data.csv')
data = dataset[['days_deliverd','net_price','total_units','designs_prior_30','errors','sales_bulk_following_365']]
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

## Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Training the Decision Tree Regression model on the Training set

In [21]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(X_train, y_train)

DecisionTreeRegressor(random_state=0)

## Predicting the Test set results

In [22]:
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[ 460.64  234.3 ]
 [ 328.48  277.75]
 [ 986.28  272.14]
 ...
 [ 513.06  828.24]
 [ 206.56 3610.11]
 [5921.1  1779.  ]]


## Evaluating the Model Performance

In [23]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

-0.47186291830950844